In [1]:
from flask import Flask, render_template, request, redirect, url_for
from emot.emo_unicode import UNICODE_EMOJI
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re

In [2]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

def remove_punct(x):
    import string
    punct_tag=set(string.punctuation)
    t=[i for i in x if i not in punct_tag]
    return t

def tokenize(x):
    return word_tokenize(x)

def whites(x):
    w={' '}
    x=[i for i in x if i not in w]
    return x

def re_stop(x):
    nltk_stopwords = nltk.corpus.stopwords.words("english")
    return [ token for token in x if token not in nltk_stopwords]

def remove_num(data):
    data=' '.join(data)
    tag=re.compile(r'[0-9]+')
    data=tag.sub(r' ',data)
    return data

def remove_roman(data):
    en_tag =re.compile(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$')
    data=en_tag.sub(r' ',data)
    return data

def remove_redun(data):
    red_tag=re.compile(r'[?<=(  )\\]|[&&|\|\|-]')
    data=red_tag.sub(r' ',data)
    data=data.split(" ")
    return data


def stemm(x):
    l=[]
    for i in x:
        l.append(PorterStemmer().stem(i))
    return l  



In [13]:
text = "esteemed mr lebedev lady gentleman gathered sum result past year sure end discussion year russia turned new page history right time openly discus change taken place judicial system last decade look like near future legal professional judge always enjoyed special prestige praised measured approach word action hand law acquires real power becomes instrument upholding justice acting behalf state hand verdict behalf russian federation court law mean power enormous power entail huge responsibility usual view court disciplinary machine giving speedy always fair judgment confined past habit seeking protection superior replaced people ’ mind understanding convenience advantage taking matter court defending right assistance today citizen willingly frequently judicial system court live higher expectation going talk good opportunity ponder current condition judicial system present  ,  criminal civil case awaiting trial russian court almost  ,  case tried moscow court alone january  . past year  ,  case returned investigation one third verdict delayed regrettably figure show busy judge also many people still waiting fair resolution problem meanwhile many case problem determine future fully aware defendant spending month sometimes year detention facility hearing civil case involving housing inheritance family issue delayed age meanwhile people judging effectiveness government power integrity court level democracy worldly matter reason huge liability judiciary country ’ leader therefore government authority whole judiciary ’ real independence best indicator state based rule law old universal problem old world time anatoly koni said judge “ protected condition conducive cowardice forced assentation. ” hit nail head regrettably witnessing relapse old attitude court appendage bureaucratic machine instance russian president ’ prerogative delegate power judge context alarming regional leader trying place judicial system control everyone remembers ingush leader ’ attempt hold republican referendum procedure appointing judge violation federal law guarantee equal right citizen throughout russia moreover constitution russian republic still contain clause resident appointed judge formula dozen transgression would like say right away fully support position supreme court russian federation ’ council judge oppose encroachment existing procedure forming court appointing federal judge equally talk revising judiciary ’ current status imposing limit guarantee immunity independence autonomy sanctioned law attempt transgress upon constitutional principle nipped bud eight year ago supreme soviet russian federation approved judiciary reform concept aimed creating strong independent legal system judiciary took direct part process renewal supreme court initiated adopted law military tribunal judicial department funding court russian federation importance adopted legislation judicial system still ridden many serious problem many task still ahead first legal foundation administering justice full accordance russian constitution unjustified delay adoption basic law court law new code criminal legal proceeding legislative act administrative proceeding court badly need work russia ’ constitutional court already issued   resolution discrepancy individual provision code criminal proceeding constitution general message resolution reinforce adversary proceeding maybe time implement full constitutional rule whereby person may arrested detained basis court ruling mean right give serious thought second old rule provide effective legal protection lawful right interest citizen particular rule related providing broader access justice defence preliminary investigation finally russian legislation reformed line universally recognized norm international law know may     russia ratified convention protection human right fundamental freedom become part legal system addition recognized jurisdiction european court human right today must pay special attention problem russian judicial system european court may object first applies meeting reasonable deadline court deliberation european court accept excuse overload lack judge shortage fund national court drag trial today would like discus another key aspect activity – case involving economic crime transition centralized planned economy one governed market impossible without strengthening foundation latter primarily effective performance legal institution regulating economy state primarily guarantee market ’ streamlined functioning stable environment demand fully applies legislative executive branch government also court law arbitration new civil criminal code combine two approach protection right lawful interest participant economic relation one hand guarantee entrepreneur ’ freedom economic performance right hand interest state demand economic agent well role guarantee interest participant promoting tax collection well combating money laundering illegal export capital would like stress one point government heavily criticized abandoning economy without reason already mentioned several time believe repeat talking colleague state interfere economy case resurrect command based planned economy instead create powerful lever understandable rule enforce addition regulating economy state make sure rule everyone guarantee market player follow case transparent attractive investment environment needle say state create new mechanism also keep old one good shape directly applies funding judicial system recently question became urgent russia ’ constitutional court say article   constitution read “ court law shall financed federal budget financing shall ensure full independent administration justice accordance federal law. ” concern funding programme development judicial system also financial support judge receiving modest salary dealing case worth billion rouble fighting crime dangerous offender without proper protection reality today give much reason optimism go without saying everything comply state ’ ability also clear state want everything function normally productive economy effective institution must pay attention want make promise would wrong speak big rise salary would like know understand rise necessary certainly move direction must say much already done give judge due last year judge court administration official received full salary time funding increased almost   million rouble order raise official rate pay half battle programme strengthen rule law combat crime adequately funded many law already passed enforced lack money defendant ’ constitutional right trial jury put practice reason today trial jury limited nine region russia operating everywhere finally sometimes lack money court even meet basic need summon witness interpreter expert law demand enough form envelope money mailing expense still enough computer technical facility court record shirk responsibility situation work improve already said much effect think state fails pay proper attention problem able collect tax owed economy perform well therefore tight fisted court meanwhile court also take step control expense streamline cash flow otherwise difficult resist attempt local leader “ privatize ” court partial funding one major problem want point personnel already said complaint overload court surfaced often become commonplace indeed still  ,  vacancy judge year authorised staff size increased thousand position presidential decree federal judge relieved part burden implementation federal law justice peace almost  ,  people sworn justice plan making even urgent create effective system training proper screening judge essential improve screening process also streamline rule appointing judge removing office current procedure complicated take extremely long time guarantee promotion would also like say personnel problem limited described authority judicial power effective performance largely determined judge ’ qualification unbiased attitude needle say average two three time fewer judge per caput many western country deny citizen right speedy trial friend know well result work direct effect political economic life society court remain key factor safeguarding democracy allowing russia forge ahead new century received lot extra work since december election citizen association alerted violation electoral law federal regional body government flood complaint likely subside beginning summer know position – mean government ’ position state society need honest election result open honest election campaign way achieving fully fledged effective government way guaranteeing democracy order conference certainly talk much implementation judicial reform let ’ tell straight – reform dragged difficult society looking ambiguously human faith inviolability law depends final result personal decency professionalism much hope continue guided work principle professional honour judicial duty responsibility homeland would like wish russia ’ judiciary best thank attention"
text = text.lower()
text = convert_emojis(text)
text = tokenize(text)
text = remove_punct(text)
text = whites(text)
text = re_stop(text)
text = remove_num(text)
text = remove_roman(text)
text = remove_redun(text)
text = stemm(text)
text = ' '.join(text)

In [17]:
type(text)

str

In [14]:


tfidf_filename = "./tfidf_vectorizer.pkl"
loaded_tfidf = joblib.load(tfidf_filename)

C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
text = [text]


In [19]:
model = pickle.load(open('./your_model.pkl', 'rb'))
result = model.predict(x)
result

C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([3], dtype=int64)

In [21]:
print(result[0])

3


In [ ]:
print("---------------------------------------------------------------------")
    print(text)
    
    
    
    if result==0:
        return render_template('result.html', text = "JOY")
    elif result==1:
        return render_template('result.html', text = "NETURAL")
    elif result==2:
        return render_template('result.html', text = "OPTIMISM")
    else:
        return render_template('result.html', text = "UPSET")